# Introducing attention briefly
<br>
<br>
<br>
<br>
<br>
<br>


## Study examples of attention

<img src="attention_ex1.PNG" width="800" align="center"/> 

<img src="attention_ex2.PNG" width="800" align="center"/> 

<img src="attention_ex3.PNG" width="1000" align="center"/> 

***

<span style="font-size:0.5em;"> 
[참고] A STRUCTURED SELF-ATTENTIVE SENTENCE EMBEDDING  
       Show, Attend and Tell: Neural Image Caption Generation with Visual Attention </span>

## Sequence2sequence problems
ex) 번역, 챗팅, 문서, 이미지 캡션, ..., (주소 --> 우편번호)

#### Encoder-Decoder model
Seq2seq 문제 해결에 효과적이며 널리 쓰임

#### Structures of Attention in encoder-decoder
0. RNN wo attention
1. RNN + attention
2. Attention wo RNN

## Simple encoder-decoder model
<img src="encoder_decoder.PNG" width="800" align="center"/> 
* 두 개의 RNN(encoder, decoder) 이 연결된 형태 : SimpleRNN, LSTM, GRU
    * input : (batch_size, timesteps, input_dim)
    * output : 
        * (batch_size, timesteps, units) if return_sequences
        * (batch_size, units) if return_state
* 예) 뒷 단어 Y = f(W, X, (A, B, C))  

## Simple encoder-decoder with Sequential

Sequential로 모델을 구성하는 경우 encoder의 출력 값을 decoder 의 입력값에 맞춰주어야 함

```python
# define model
model = Sequential()
model.add(LSTM(150, input_shape=(n_timesteps_in, n_features)))
model.add(RepeatVector(n_timesteps_in))
model.add(LSTM(150, return_sequences=True))
model.add(TimeDistributed(Dense(n_features, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
```

[참고] https://machinelearningmastery.com/encoder-decoder-long-short-term-memory-networks/

## Simple encoder-decoder with Model

Model 로 모델을 구성하는 경우 encoder의 최종 상태를 decoder 의 시작 상태로 넣어 줌
```python
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
```
[참고] https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

## Another encoder-Decoder model
<img src="encoder_decoder2.jpg" width="600" align="left"/>  

* Encoder로 얻은 context를 decoder 초기값과 이후 시점 계산에도 더해줌
    * decoder 진행에 따른 처음 정보의 손실 감소

## Attention model

#### 기존 모델 단점 :
* context 로 sentence의 모든 정보 집약 후 이로부터 새롭게 문장 생성
    * 여러 단어들의 조합인 문장이 단어 하나와 동일한 크기의 벡터 내에 저장될 수 있을까?  
    * sentence1 | encoder | --> context --> | decoder | sentence1p
*  문장이 길어질수록 성능 나빠짐  

#### Attention 추가를 통한 보완
* (기존) 한 개 context vector와 연결 --> 전체 입력 hidden state 와 연결
* <span style="color:red"> decoder 각 요소와 encoder 각 요소간 연관성 확인 가능</span>

<img src="encoder_decoder3.jpg" width="400" align="left"/>  


언어모델 : 현재까지의 문장으로부터 다음 단어 예측,  
$ p(y_i|y_1, ..., y_{i-1}, x)$   
<br>
$ p(y_i|y_1, ..., y_{i-1}, x) = g(y_{i-1}, s_i, c_i) $  
$ s_i = f(s_{i-1}, y_{i-1}, c_i) $  
$ \hspace{0.7cm} c_i = \sum_{j=1}^{T_x} \alpha_{ij} h_j $  
$ \hspace{1.4cm} \alpha_{ij} = \frac{\exp(e_{ij})}{\sum_{k=1}^{T_x} \exp(e_{ik})} $  
$ \hspace{1.4cm} e_{ij} = a(s_{i-1}, h_j) = v_a^T \tanh(W_a s_{i-1} + U_a h_j) $  
$ g, f, a $ 는 데이터로부터 학습할 함수, 사용자가 설정 가능     

## Attention 예시

#### Toy example
x1, x2, x3 로 구성된 sequence 가 있는 경우   

```python  
h1, h2, h3 = Encoder(x1, x2, x3)

# a는 데이터로 부터 학습할 함수, 예) feed forward network
e11 = a(0, h1) # h : encoder 중간 상태
e12 = a(0, h2)
e13 = a(0, h3)

e21 = a(s1, h1) # s : decoder 중간 상태
e22 = a(s1, h2)
e23 = a(s1, h3)

alpha11 = exp(e11) / (exp(e11) + exp(e12) + exp(e13))
alpha12 = exp(e12) / (exp(e11) + exp(e12) + exp(e13))
alpha13 = exp(e13) / (exp(e11) + exp(e12) + exp(e13))

alpha21 = exp(e21) / (exp(e21) + exp(e22) + exp(e23))
alpha22 = exp(e22) / (exp(e21) + exp(e22) + exp(e23))
alpha23 = exp(e23) / (exp(e21) + exp(e22) + exp(e23))

c1 = alpha11 * h1 + alpha12 * h2 + alpha13 * h3
c2 = alpha21 * h1 + alpha22 * h2 + alpha23 * h3

s1 = Decoder(c1)
s2 = Decoder(s1, y1, c2)
```

## Attention 예시 (계속)

#### RNN vs Attention
* 문제 : 랜던한 수열에 대한 학습
    * [1,3,5,2,4] --> [1, 3]
    * 랜덤 수 이므로 실제 앞 뒤 숫자간 규칙 없음 : RNN은 찾기 어려움
    * Attention 은 입력, 출력의 관련성을 확인하므로 복사되는것 알아냄  
* [attention 구현 코드](attention_layer.ipynb) : https://github.com/datalogue/keras-attention  

#### attention layer 적용

```python
# define the encoder-decoder with attention model
def attention_model(n_timesteps_in, n_features):
	model = Sequential()
	model.add(LSTM(150, input_shape=(n_timesteps_in, n_features), return_sequences=True))
	model.add(AttentionDecoder(150, n_features))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
	return model
```

#### attention 정의 함수의 호출 구문 (step_function)

```python
# https://keras.io/backend/
keras.backend.rnn(step_function, inputs, initial_states, go_backwards=False, mask=None, constants=None, unroll=False, input_length=None)
# Iterates over the time dimension of a tensor.
...
```

#### attention 정의 함수

```python
# in class AttentionDecoder
def step(self, x, states):
    ytm, stm = states

    # repeat the hidden state to the length of the sequence
    _stm = K.repeat(stm, self.timesteps)

    # now multiplty the weight matrix with the repeated hidden state
    _Wxstm = K.dot(_stm, self.W_a)

    # calculate the attention probabilities
    # this relates how much other timesteps contributed to this one.
    et = K.dot(activations.tanh(_Wxstm + self._uxpb),
               K.expand_dims(self.V_a))
    at = K.exp(et)
    at_sum = K.sum(at, axis=1)
    at_sum_repeated = K.repeat(at_sum, self.timesteps)
    at /= at_sum_repeated  # vector of size (batchsize, timesteps, 1)

    # calculate the context vector
    context = K.squeeze(K.batch_dot(at, self.x_seq, axes=1), axis=1)

```

[참고]  
https://machinelearningmastery.com/how-does-attention-work-in-encoder-decoder-recurrent-neural-networks/  
https://machinelearningmastery.com/encoder-decoder-attention-sequence-to-sequence-prediction-keras/    

## Transformer : Advanced attention model

위의 encoder-decoder 구조에서 RNN 없이 attention 만으로 구성

#### RNN 단점
* 속도 저하
    * 한 단어씩 차례대로 진행함에 따라 병렬 처리 어려움
* 문장이 길어질수록 성능 저하

#### Transformer 특징 
* 각 단어에 대해 별개로 처리하므로 병렬 처리 가능
* attention을 이용해 긴 문장에 대해서도 주변 단어들에 대한 영향 반영하여 성능 향상  
* __<span style="color:red"> Multi-head attention </span>__  

<br>  
참고  
* https://arxiv.org/abs/1706.03762 (Attention is all you need)  
* http://nlp.seas.harvard.edu/2018/04/03/attention.html (코드, pytorch 이나 이해하기 쉬움)  
* https://github.com/Separius/BERT-keras/tree/master/transformer (코드)    

## Attention in Transformer

* attention 일반화 : (기존) s, h --> Q, K, V 로 이해
    * 의미 : Query가 주어졌을 때 Key와 비교해 연관성 확인 후 해당 Value에 가중치
* 예) 영문 --> 한글문으로 번역하는 경우
    * I love you --> 나는 너를 사랑해
        * (I love you, 나는 너를) --> (사랑해)
        * query : 나는 너를
        * key, value : I love you
        * "사랑해"는 (key, value) 에 대한 query 의 관련성으로 부터 확인 가능

## Transformer 구조
<img src="transformer1.jpg" width="700" align="center"/>  

## Encoder, decoder in Transformer

#### 구성 
* (embedding, positional encoding, attention, normalization, feed foward) 반복

#### encoder, decoder 차이
* encoder : self-attention (input 문장만 사용)
* decoder : (masked) self-attention, encoder-decoder attention
* 예) 영문 --> 한글문
    * encoder : 영문에 대한 self-attention 
    * decoder : 
        * 한글문 output 앞부분에 대한 self-attention --> query
            * mask 를 적용해 각 예측 단어보다 먼저 나온 단어들만 실제 계산에 참여
        * 영문 encoder 결과를 (key, value) 로 확인해 처음 영문 및 앞단의 한글 영향을 고려하여(query, key, value) 다음 한글 단어 예측
> mask  
      * 번역되는 문장은 한 단어씩 차례로 생성하여 전체 문장 완성   
          * I love you(0) : 나는(1) --> 너를(2) --> 사랑해(3)
      * 아직까지 나오지 않은 단어에 대해서는 계산에 포함하지 않음
      * 계산 병렬적으로 적용되어 계산 효율적

## Encoder, (decoder) in Transformer (계속)

#### Encoder 사용법 및 구현 부분

```python
x = embedding_layer(inputs)
for i in range(num_layers):
    x = EncoderLayer(embedding_dim, num_heads, d_hid, residual_dropout,
                     attention_dropout, use_attn_mask, i, neg_inf, layer_norm_epsilon, accurate_gelu)(x, attn_mask)
return keras.Model(inputs=inputs, outputs=[x], name='Transformer')

class EncoderLayer:
    def __call__(self, x, mask):
        a = self.attention(x, mask)
        n = self.ln1(self.add1([x, self.drop1(a)]))
        f = self.ffn(n)
        return self.ln2(self.add2([n, self.drop2(f)]))
```      

https://github.com/Separius/BERT-keras/blob/master/transformer/model.py    

## Multi-head attention

<img src="multi_head_ex.PNG" width="800" align="center"/> 

## Multi-head attention(계속)

<img src="transformer2_3.PNG" width="600" align="center"/>  

$ Attention(Q, K, V) = softmax(\frac{QK^T}{\sqrt{d_k}}V) $  
위에서의 $a$ 함수에 대하여 내적함수 형태로 단순하게 적용
대신 CNN 에서 다수의 filter 를 적용하는 것처럼, 여러 개로 나누어 적용 
* 각 Q, K, V 데이터를 n_head 개 만큼 split
* 각 데이터에 선형 transformation 적용
* 내적 적용
* 값들을 concatenate 해서 split 되기 이전으로 사이즈로 복원

```python
def multihead_attention(x, attn_mask, n_head: int, n_state: int, attention_dropout: float, neg_inf: float):
    _q, _k, _v = x[:, :, :n_state], x[:, :, n_state:2 * n_state], x[:, :, -n_state:]
    # 분리
    q = split_heads(_q, n_head)  # B, H, L, C//H
    k = split_heads(_k, n_head, k=True)  # B, H, C//H, L
    v = split_heads(_v, n_head)  # B, H, L, C//H
    a = scaled_dot_product_attention(q, k, v, attn_mask, attention_dropout, neg_inf)
    return merge_heads(a)

# 내적 부분만 살펴보면
def scaled_dot_product_attention(q, k, v, attn_mask, attention_dropout: float, neg_inf: float):
    w = K.batch_dot(q, k)  # w is B, H, L, L
    w = w / K.sqrt(K.cast(shape_list(v)[-1], K.floatx()))
    if attn_mask is not None:
        w = attn_mask * w + (1.0 - attn_mask) * neg_inf
    w = K.softmax(w)
    w = Dropout(attention_dropout)(w)
    return K.batch_dot(w, v)  # it is B, H, L, C//H [like v]
```

https://github.com/Separius/BERT-keras/blob/master/transformer/funcs.py

## 참고하면 좋을 것
1. Keras code : https://github.com/Lsdefine/attention-is-all-you-need-keras/blob/master/transformer.py  
2. Attention is all you need : https://arxiv.org/pdf/1706.03762.pdf  
3. 좋은 설명 : https://jalammar.github.io/illustrated-transformer/  
4. 뉴스 분류 응용 : [Hierarchical Attention Networks for Document Classification](https://www.cs.cmu.edu/~./hovy/papers/16HLT-hierarchical-attention-networks.pdf)